### Tips for DL

Deep learning的常规流程

![](./ml_png/6_normal_dl_process.png)

一些对应的改进措施：

①，在Training data上表现差两个应对方案：

    A. New Activation Function
    B. Adaptive learning rate
    
②，在Testing data上表现出的三个应对方案：

    A. Early stopping
    B. Regularization
    c. Dropout
  

#### New activation function

问题：为什么要用新的？原来的sigmoid activation function出了什么问题？有哪些新的activation function?

回答：随着层数的不断加深，旧的sigmoid function 会引起Vanishing gradient problem.考虑梯度值

$$
\frac{\partial L}{\partial w_i} = \frac{\partial z}{\partial w_i}· \frac{\partial L}{\partial z} = 
 \frac{\partial z}{\partial w_i} \left(  \frac{\partial a}{\partial z}· \frac{\partial L}{\partial a}  \right) = 
 x_i · \sigma'(z)· \left[  w_4 ·  \frac{\partial L}{\partial z'}  +  w_5· \frac{\partial L}{\partial z''}  \right]
$$

其中$\sigma'(z) = \sigma(z) \left(  1-\sigma(z)  \right), \ \ 0<\sigma(z) < 1$. 如果$z'$和$z''$后面还有上百层的$z^{i}$,那么整个由多个$\sigma$构成的数值将会变得非常小，如$0.99^{100} \approx 0.3660$, 当基底变小，有$0.9^{100} \approx 2.656\times 10^{-5}$.此时整个gradient会变得非常小。进行gradient descent时，有
 
$$
w_1 = w_0 - \eta \frac{\partial L}{\partial w_0}
$$

其中$w_0$为初始的随机化weight参数。在反向传播时，越靠近output的gradient由于相乘的$\sigma$较少，故$\frac{\partial L}{\partial w_N} $较大，学习较快，收敛的速度也会更快。而随着backward pass的不断进行，越靠近Input的gradient,如$w_1$越小，参数学习的速度也相对更慢，可能而$w_1$还在缓慢地向极值靠近，$w_N$就已经根据前面的$w$值收敛到某个极值附近。考虑到所有weight参数都是随机初始的，前面的weight值随机且更新很慢，而后面的weight值已经收敛。极端情况下，靠近input的weight值在随机初始化的初始值附近缓慢更新，而靠近output的weight已经停止更新。此时总的参数更新已经相当缓慢，收敛到某loss值附近，会被误认为已经到达极值附近并发出停止training的错误信号.

正是由于这种梯度缓慢消失的情况出现，因此sigmoid被认为不合理，矛盾点在于激活函数的多次相乘所得到的值趋向于零。

另一种说法，为什么sigmoid function有问题的解释是，如下图。

![](./ml_png/6_relu_reason.png)

求偏微分的过程可以简单地理解为自变量的变化（$\Delta w$）对因变量值的变化（$\Delta l$）大小，直觉上我们期望这个值大些。因为它反映了这个model的灵敏程度。比如将model用于猫狗分类时，如果loss对输入的dog,cat图片的响应相差不大，那么一定程度上代表这个model很难对不同的class进行分类。而应用sigmoid function时，正是这种情况，即loss对weight的变化不敏感。从上右图sigmoid function两组input值和两组Ouput值可反应出该问题。尤其是当layer不断加深而这种效应的不断叠加，loss前和loss后的值将相差不大。

为了缓解梯度消失的问题，第一个解决方案是activation function更改为ReLU。

![](./ml_png/6_relu.png)

ReLU的特性。在输出的$z = \sigma(w_i x_i + b) <0$ 时，$a=0$; 而$z>0$时，$a=z, a'=1$.此时，从$\frac{\partial L}{\partial w_i}$公式的角度便不再出现多个$a'=\sigma'(z)$的相乘会导致gradient消失的情况。因为$a$值不再在$0<a<1$的范围。

如果从Neural Network Structure的角度来看。则可能存在如下可能性。

![](./ml_png/6_relu2.png)

如果Layer1中的$z_1^3,z_1^4<0$，则$a_1^3,a_1^4=0$,相当于这个neuron被抛弃，同理Layer2中$z$值小于零时有同样的效果。最终去除了$z_1^3,z_1^4,z_2^1, z_2^3$所在的neurons的NN如上图右图示。该过程与overfitting过程的dropout类似（dropout需要定义dropout rate，这里不需要，因此灵活度更大）。分析最终形成的thinner linear network，其中的neurons大幅度减少，总体参数量减少。靠近input的weight的smaller gradients的情况得到缓解，因为全连接权重被压缩为原来的一半，计算gradient需要累乘的$\sigma'(a)$个数减少为原来的一半。

ReLU的变体有以下两种，它们都是针对$z<0$的情况而言，如下图

![](./ml_png/6_leakyrelu_prelu.png)

其中Leaky ReLU在$a$轴左侧有斜率$\alpha=0.01$，parametric ReLU中左侧的斜率值$\alpha$是学习得到的。

除了ReLU以外，还有一种激活函数也是常用的,即Maxout。该激活函数的动态的，被称为learnable activation function.这里的maxout就是输出包含了多个$z$的one group中的最大值。如下图

![](./ml_png/6_maxout.png)

这里的一个neuron包含了$z=\sum_i w_i x_i +b$和$a=max\{ z_i^1,z_i^2 \}$两个变量的求值。Maxout可视为ReLU的更一般形式，或者说，ReLU是Maxout的特殊形式。如下图

![](./ml_png/6_maxout_relu.png)

左侧是对$z=w x +b$进行ReLU运算，可画出左下角的折线图，其中$z$小于零的地方置零；右侧是比较出直线$z_1=w x +b$和直线$z_2=0 \times x +0 \times 1$之间的最大值$a= \max{z_1, z_2}$，然后取他们的最大值作为输出。需注意的是，Maxout还可以表示除ReLU以外的其他激活函数。分别用$w',b'$来取代上面右侧图的$0,0$,则分别有直线$z_1 = wx+b$和$z_2=w'x+b'$，如下图

![](./ml_png/6_maxout_morethan_relu.png)

总结：

1. 在Maxout Network中，该Maxout function可以是任意的分段线性凸函数；

2. “分段”中的段数取决于一组中有多少个$z$值。上面的几幅图中$z$值为2，所有activation function被分为了两段。除此以外，还有可能分三段及以上，如下图所示

    ![](./ml_png/6_maxout_group.png)

3. Maxout的训练过程分为三步，第一步是先找到每个neuron的最大值；第二步是去除非最大值所在的neuron；第三步是训练剩余的network,如下图。

    ![](./ml_png/6_maxout_eventually2.png)

#### Adaptive Learning Rate

最开始的参数更新策略为随机梯度下降算法SGD，具体的参数更新流程如下

1. 计算$\nabla L(\theta^0)$, 更新参数$$\theta^1 = \theta^0 - \eta \nabla L(\theta^0)$$

2. 计算$\nabla L(\theta^1)$, 更新参数$$\theta^2 = \theta^1 - \eta \nabla L(\theta^1)$$

3. 计算$\nabla L(\theta^2)$, 更新参数$$\theta^3 = \theta^2 - \eta \nabla L(\theta^2)$$

    $\vdots$

4. 计算$\nabla L(\theta^{t-1})$, 更新参数$$\theta^t = \theta^{t-1} - \eta \nabla L(\theta^{t-1})$$

提问：SGD在参数更新过程中有什么缺点，有哪些做法可以弥补之？

回答：在loss较平缓的地方，gradient趋于零。理想凸函数中，gradient接近零的地方被为是grobal minima.但实际上loss可能凹凸不平、坑坑洼洼，即gradient接近于零的地方可能是高原 /saddle point / local minima等。而在物理世界中，当我们让一个小球从一个滑坡处滚下，当gradient$\approx 0$时，它并不会停下来，而是会继续往先前的运动方向继续滑动一段距离。把这个现象引入到参数的更新过程中去，于是就是SGDM,这里的M指的是动量Momentum。

##### Momentum

Momentum的计算过程如下

1. $v^0 = 0$;

2. 计算$\theta^1 = \theta^0 + v^1$, 其中$$v^1 = \lambda v^0 - \eta \nabla L(\theta^0)$$

3. 计算$\theta^2 = \theta^1 + v^2$, 其中
$$\begin{align}
v^2 &= \lambda v^1 - \eta \nabla L(\theta^1)   \\
&= \left( \lambda^2 v^0 - \lambda^1 \eta g^0  \right) - \lambda^0 \eta  g^1
\end{align}$$

3. 计算$\theta^3 = \theta^2 + v^3$, 其中
$$\begin{align}
v^3 &= \lambda v^2 - \eta \nabla L(\theta^2)  \\
&= \left( \lambda^3 v^0 - \lambda^2 \eta g^0 - \lambda^1 \eta g^1 \right) - \lambda^0 \eta  g^2
\end{align}$$

    $\vdots$

4. 计算$\theta^t = \theta^{t-1} + v^t$, 其中

$$\begin{align}
v^t &= \lambda v^{t-1} - \eta \nabla L(\theta^{t-1})  \\ 
&= \left( \lambda^t v^0 - \lambda^{t-1} \eta g^0 - \lambda^{t-2} \eta g^1 - \lambda^{t-3} \eta g^2 - \dots - \lambda^1 \eta g^{t-2}\right) - \lambda^0 \eta g^{t-1}
\end{align}$$

可见，在更新参数值时，不仅仅考虑梯度的方向，还考虑了过去梯度方向汇总的总方向，其中后者我们视其为动量，或者物理环境中的惯性。

![](./ml_png/6_momentum.png)

#### Adagrad

在实际的NN训练过程中，我们总是希望learning rate是随着Epoch的增大而不断变小，以保证loss稳定地落在在minima而不发生强烈的振荡。而上述的SGD和SGDM其learning rate都是固定不变的$\eta$，因此，需要找到一个方法使lr能进行自适应地改变。第一种是Adagrad方法，它的数学表示如下

$$
w^{t+1} \leftarrow w^t - \frac{\eta}{\sqrt{\sum_{i=0}^{t}{(g^i)}^2}}g^t
$$

物理意义：在NN中，参数的更新与否通常不是固定的、同步的。在一个Epoch中，可能参数$w_1$被更新，而参数$w_2$不被更新，亦即某些参数在总的Epoch中，可能被更新得频繁些，某些参数的更新频率低些。那么，对于那些被频繁更新的参数，我们在它的身上下了较多的功夫，对它进行了较多的研究并积累了有关的参数选择的丰富知识，这些知识都是通过大量的训练数据习得的。当再次输入一个新样本用于训练时，并不希望该单样本对这个weight参数有很大的影响，即希望该weight的更新速率能慢一些。反之对于那些不被经常更新的参数希望它一旦被更新，其更新速率稍快些。

在这里，更新速率的快与慢主要受学习率影响。上式中，$\sqrt{\sum_{i=0}^{t}{(g^i)}^2}$是对过去gradient的值的累加。它单调递增，故整体的lr是越来越小的。另一方面，weight被更新得越频繁，那么它的weight值被累加的次数越多，lr越小。反之越大。

#### RMSProp / AdaDelta

成也萧何败也萧何，随着$\sqrt{\sum_{i=0}^{t}{(g^i)}^2}$越来越大，lr将逐渐趋近于零。但如果后续仍有新的training data,那么由于lr过小，NN很难去拟合这些新的data，相当于这些辛苦收集得来的data被白白浪费，甚至影响后面的test精度。归根结底，是因为Adagrad将过去所有的gradient都考虑进来了。如果只将过去某一个时间段内的gradient纳入考虑，是否能解决这个问题？这就是AdaDelta致力于解决的问题，这里的Delta就是指考虑的过去的gradient的比例。它的具体算法如下。

$$\boxed{\begin{array}{rl}
计算g^0，w^1 \leftarrow w^0 - \frac{\eta}{\delta^0}{g^0}  , & \delta^0 = g^0 \\
计算g^1，w^2 \leftarrow w^1 - \frac{\eta}{\delta^1}{g^1}  , & \delta^1 = \sqrt{\alpha \left( \delta^{0} \right)^2 + \left( 1-\alpha\right) \left( g^{1} \right)^2} \\
计算g^2，w^3 \leftarrow w^2 - \frac{\eta}{\delta^2}{g^2}  , & \delta^2 = \sqrt{\alpha \left( \delta^{1} \right)^2 + \left( 1-\alpha\right) \left( g^{2} \right)^2} \\
\vdots & \vdots \\
计算g^t，w^{t+1} \leftarrow w^t - \frac{\eta}{\delta^{t}}{g^t} , & \delta^t = \sqrt{\alpha \left( \delta^{t-1} \right)^2 + \left( 1-\alpha\right) \left( g^{t} \right)^2} \\
\end{array}}$$

#### Adam

Adam将同时将动量Momentum和Adaptive learning rate考虑进来，是AdaDelta+Momentum的组合。


#### Early Stopping

以上讨论了在Training process过程中Accuracy不理想的情况下的应对措施。那么如果Training精度足够高，但将训练模型套用到Testing时，效果不理想，又应该如何去调整NN结构呢？第一种方法是提前结束训练。这里面蕴含的道理是，随着Training的逐渐深入，模型参数构成的函数会首先拟合出给定训练数据的一半规律，随后拟合出它的特殊规律。这与linear regression中从低阶到高阶演变的过程类似。为了避免拟合函数演化到过高阶次，提前结束这个过程是一个可取的办法。那么应该以什么为标准来作为停止的最合适时机呢？常常需要借助Validation来辅助。在Training中每经过一个Epoch，就在Validation data上验证它的accuracy和测量出Loss。经过多个Epoch,能画出一条Loss变化曲线或accuracy 曲线。如下图，只需要在loss整体趋势增大的epoch提前结束训练即可。

![](./ml_png/6_early_stopping.png)


#### Regularization

正则化的方法前面已经讲解过，它的具体做法是在原损失函数的损失项（MSE或cross entropy）的基础上附加一个正则项，目的是保证$\hat{y}$和$y$的距离较小的同时，各个weight的值尽可能小。这一方面能保证输入数据（尤其是输入包含了噪声数据）的变化对输出的影响较小，另一方面也能避免函数向过高阶数发展。加入了regularization的loss函数如下

$$
L'(\theta) = L(\theta)+\lambda \frac 12 \Vert  \theta \Vert_2
$$

其中的$\theta=\{ w_1, w_2, \dots\}$是NN中所有$w,b$构成的参数，$\Vert  \theta \Vert_2$为$L_2$正则项,

$$
\Vert  \theta \Vert_2 = (w_1)^2 + (w_2)^2+ ...
$$

既然已经有了loss函数，那么下一步就是通过求gradient来更新参数。由于

$$
\frac{\partial L'}{\partial w} = \frac{\partial L}{\partial w} + \lambda w
$$

则有

$$\begin{align}
w^{t+1} \leftarrow - \eta \frac{\partial L'}{\partial w} &= w^t - \eta \left(  \frac{\partial L}{\partial w} + \lambda w^t \right)  \\
&= w^t - \eta \lambda w^t - \eta \frac{\partial L}{\partial w} \\
&= \left( 1- \eta \lambda  \right) w^t - \eta \frac{\partial L}{\partial w}
\end{align}$$

此时前面的Weight将会小于1，此即weight decay，这与上面讨论的目标是一致的，即weight是不断趋于零的。

除了使用$L_2$正则化，也常常使用$L_1$正则化，即

$$
L'(\theta) = L(\theta)+\lambda \frac 12 \Vert  \theta \Vert_1, \mathrm{where} \ \Vert \theta \Vert_1 = \vert w_1 \vert + \vert w_2 \vert + \dots
$$

对应的参数更新过程为

$$\begin{align}
w^{t+1} \leftarrow  w^t - \eta \frac{\partial L'}{\partial w} &= w^t - \eta \left(  \frac{\partial L}{\partial w} + \lambda  \mathrm{sgn}( w^t) \right)  \\
&= w^t - \eta \lambda  \mathrm{sgn}( w^t)  - \eta \frac{\partial L}{\partial w} \\
&\approx  \left( 1- \eta \lambda  \right) w^t - \eta \frac{\partial L}{\partial w}
\end{align}$$

该过程与$L_2$正则化类似。

#### Dropout Training

Dropout前面也已经陈述过，具体就是在训练阶段以$p \%$的概率删去每个Layer的neuron，而在测试阶段保持NN结构不变，对每个weight与$(1-p)\%$相乘。

为什么用dropout能起作用呢？直观解释如下

1. 如果把参数的学习过程比作一个team完成一项任务。team中的各个组员天然就有懒惰的倾向，认为就算自己不努力，组内的其他人也会想方设法地完成任务。往组员内部插播一条带有不确定性的消息（比如指导老师说他知道有一位组员不积极参与讨论，且如果他继续这么做这门功课的分数为零），那么各个成员之间的积极性将被调动起来，尽己所能做到最好。

2. 而测试过程中不使用dropout，相当于评分时不抛弃任何一个人，认为他们付出得一样多。比如要对这项任务完成presentation才进行最终评分，那么任意抽取其中一个人都能很好地进行细节的讲解。或者说，这个队伍中演示阶段不如意的可能性被极大压缩，因为准备阶段大家都做了很多工作。

具体回到NN中来，它相当于在一个NN结构中实现了多个NN的功能。如下图所示为training中dropout rate为$50\%$的多个NN.

![](./ml_png/6_ensemble.png)

而在testing阶段，相当于对所有的这些NN进行了一次average。

![](./ml_png/6_ensemble_testing.png)


### 总结

本节从validation和testing两条训练曲线出发，介绍了训练NN中用到的一些技巧。